In [143]:
import pandas as pd
import numpy as np

dframe = pd.read_csv("../message.csv")
dframe.head()

,Label,Message
0,0,go until jurong point crazy available only ...
1,0,ok lar joking wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor u c already then say
4,0,nah i don t think he goes to usf he lives aro...


**Q-1.1 From the training set, compute the total number of unique words in the set and the count
of each unique word in each message. Hence, if there are N unique words and M messages
in the training set, then the count of each unique word for all messages should result in a M × N matrix. You may want to use DataFrame and dictionary objects to accomplish this.
You may also use split() to ignore whitespace.**

In [144]:
# Finds all the uniq words.
uniq_words = set(w for message in dframe["Message"] for w in message.split(" ") if w != '')
print(len(uniq_words))

8753


In [145]:
# This function returns the MxN matrix given the dataframe.
# Where each cell represents the times a word appears in a sentence.
def get_dframe_with_train_indexes(train_data):
    resulting_frame = train_data[["Label"]].copy(deep=True)
    
    # we can also generate uniq_words from just the training set
    zero_arr = np.zeros((len(uniq_words),len(train_data)))
    for i, word in enumerate(uniq_words):
        resulting_frame[word] = zero_arr[i]
    
    for message, index_c in zip(train_data["Message"], resulting_frame.index):
        for word in message.split(" "):
            if word != '':
                resulting_frame[word][index_c] = resulting_frame[word][index_c] + 1
    return resulting_frame

word_count_dframe = get_dframe_with_train_indexes(dframe)

print(word_count_dframe[dframe["Message"][0].split(" ")[3]][0])
word_count_dframe.head()
    

<ipython-input-145-e16fcc3d0720>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resulting_frame[word][index_c] = resulting_frame[word][index_c] + 1


1.0


,Label,secs,k52,sitter,uploaded,kotees,hottest,crushes,camp,rise,...,increments,walking,shite,1013,westonzoyland,09058095201,prices,dedicated,strewn,lane
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Note: *In above function named get_dframe_with_train_indexes, since the matrix is sparse, as in out of 8k unique words, only ~10 are present in a sentence. So to avoid big calculations, we have initialized every value with zero, and only changed the value for those words which are present in the message. It makes our program faster than having to calculate the value for every word in a message.*

We see that our program outputs the value of "1.0" for a word that exists in the message, which confirms that our matrix has been calculated correctly.

**Q 1.2 Perform maximum likelihood estimation to determine the prior and class conditional probabilities of the training set (e.g. compute P(y = 1), P(y = 0), P(xi
|y = 0), and P(xi
|y = 1)) ,
where xi represents the i-th unique word. Be sure to confirm that these are indeed probabilities.**

In [146]:
# This function calculates the class probabilities, given the dataframe
def get_class_probs(word_count_data):
    # P(y=1) = #(1's) / total
    # P(y=0) = #(0's) / total

    count_y_1 = np.count_nonzero(word_count_data["Label"] == 1)
    count_y_0 = np.count_nonzero(word_count_data["Label"] == 0)

    p_y_1 = count_y_1 / len(word_count_data)    
    p_y_0 = count_y_0 / len(word_count_data)
    return p_y_1, p_y_0

p_y_1,p_y_0 = get_class_probs(word_count_dframe)
print("Probability of spam = " + str(p_y_1) + " and probability of not spam = " + str(p_y_0))


Probability of spam = 0.13406317300789664 and probability of not spam = 0.8659368269921034


In [170]:
#Number of times "free" appears in non-spam and spam respectively
print("Non-spam: " + str(len(word_count_dframe[(word_count_dframe["Label"] == 0) & (word_count_dframe["free"] != 0)].index)))
print("Spam: " + str(len(word_count_dframe[(word_count_dframe["Label"] == 1) & (word_count_dframe["free"] != 0)].index)))

Non-spam: 59
Spam: 170


In [147]:
# This function calculates the class-conditional probabilities, given the dataframe containing count of 
# each word in a message along with label
def get_conditional_probs(word_count_data):
    
    #p(xi | y=1) => p(xi,y=1) / p(y=1) => #(xi,y=1) / #(y=1)
    #p(xi | y=0) => p(xi,y=0) / p(y=0) => #(xi,y=0) / #(y=0)
    count_y_1 = np.count_nonzero(word_count_data["Label"] == 1)
    count_y_0 = np.count_nonzero(word_count_data["Label"] == 0)
    p_y_1_xi = dict()
    p_y_0_xi = dict()
    for word in uniq_words:
        p_y_1_xi[word] = np.count_nonzero((word_count_data["Label"] == 1) & (word_count_data[word] != 0)) / count_y_1
        p_y_0_xi[word] = np.count_nonzero((word_count_data["Label"] == 0) & (word_count_data[word] != 0)) / count_y_0
    return p_y_1_xi, p_y_0_xi

p_y_1_xi, p_y_0_xi = get_conditional_probs(word_count_dframe)


In [185]:
print('p(w="free" | y=0) = '+str(p_y_0_xi["free"]))
print('p(w="free" | y=1) = ' + str(p_y_1_xi["free"]) + "\n\n")

print('p(y=1|w="free") + p(y=0|w="free") should be one, Validating this below...')
print((p_y_1_xi["free"] * p_y_1) / (len(word_count_dframe[word_count_dframe["free"]!=0]) / len(word_count_dframe))\
+(p_y_0_xi["free"] * p_y_0) / (len(word_count_dframe[word_count_dframe["free"]!=0]) / len(word_count_dframe)))

p(w="free" | y=0) = 0.0122279792746114
p(w="free" | y=1) = 0.22757697456492637


p(y=1|w="free") + p(y=0|w="free") should be one, Validating this below...
1.0


**Q3: Once the above probabilities are determined, use Naive Bayes classification to classify each
of the testing examples as spam or not. Ignore words from the testing set that are not
contained in the training set. Report the accuracy, precision, recall and specificity, along
with the confusion matrix for each fold. Also report the average accuracy, precision, recall
and specificity over all folds.**

In [188]:
from sklearn.model_selection import StratifiedShuffleSplit

# This function returns laplace transformed conditional probabilities, 
# given the dataframe containing counts of each word in message
def get_laplace_conditional_probs(word_count_data):
    
    #p(xi | y=1) => p(xi,y=1) / p(y=1) => #(xi,y=1) / #(y=1)
    #p(xi | y=0) => p(xi,y=0) / p(y=0) => #(xi,y=0) / #(y=0)
    count_y_1 = np.count_nonzero(word_count_data["Label"] == 1)
    count_y_0 = np.count_nonzero(word_count_data["Label"] == 0)
    p_y_1_xi = dict()
    p_y_0_xi = dict()
    for word in uniq_words:
        p_y_1_xi[word] = (np.count_nonzero((word_count_data["Label"] == 1) & (word_count_data[word] != 0)) + 1) / (count_y_1 + len(uniq_words))
        p_y_0_xi[word] = (np.count_nonzero((word_count_data["Label"] == 0) & (word_count_data[word] != 0)) + 1) / (count_y_0 + len(uniq_words))
    return p_y_1_xi, p_y_0_xi

def predict(x_test, prior_1, prior_0, conditional_1, conditional_0, conditional_1_lap, conditional_0_lap):
    debug=True

    y_pred=[]
    y_pred_laplace = []
    for i in x_test.index:
        
        pred_1 = prior_1
        pred_0 = prior_0
        pred_1_lap = prior_1
        pred_0_lap = prior_0
        for word in x_test.columns:
            if x[word][i] == 0 or (conditional_1[word] == 0 and conditional_0[word] == 0):
                # IF not present in the current row OR it wasnt present in the training set
                continue
            
            pred_1 *= conditional_1[word]
            pred_0 *= conditional_0[word]

            pred_1_lap *= conditional_1_lap[word]
            pred_0_lap *= conditional_0_lap[word]

        y_pred.append(1 if pred_1 > pred_0 else 0)
        y_pred_laplace.append(1 if pred_1_lap > pred_0_lap else 0)
    return y_pred, y_pred_laplace

def print_results(y_pred, y_true):
    fp = tp = fn = tn = 0
    total = 0
    for pred, true in zip(y_pred, y_true):
        total+=1
        if pred == true and true == 1:
            tp+=1
        if pred == true and true == 0:
            tn+=1
        if pred != true and true == 1:
            fn+=1
        if pred != true and true == 0:
            fp+=1
    print("Accuracy: " + str((tp+tn)/len(y_true)))
    print("True Positive: " + str(tp) + " True Negative:" + str(tn))
    print("False Positive: " + str(fp) + " False Negative:" + str(fn))
    print("Precision: " + str(tp/(tp+fp)))
    print("Speficity: " + str(tn/(tn+fp)))
    print("Recall: " + str(tp/(tp+fn)))
    return (tp, tn, fp, fn)
    

def fit_and_predict(train_data, x_test, y_test):
    p_y_1, p_y_0 = get_class_probs(train_data)
    p_y_1_xi, p_y_0_xi = get_conditional_probs(train_data)
    p_y_1_xi_lap, p_y_0_xi_lap = get_laplace_conditional_probs(train_data)
            
    y_pred, y_pred_lap = predict(x_test, p_y_1, p_y_0, p_y_1_xi, p_y_0_xi, p_y_1_xi_lap, p_y_0_xi_lap)
    print("________Without_laplace_______")
    no_lap_res = print_results(y_pred, y_test["Label"])
    print("__________laplace__________")
    lap_res = print_results(y_pred_lap, y_test["Label"])
    print("\n\n")
    
    return no_lap_res, lap_res
    
x = word_count_dframe.loc[:, word_count_dframe.columns != 'Label']
y = word_count_dframe[["Label"]]

folds = StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2)

total_no_lap = (0,0,0,0)
total_lap = (0,0,0,0)
for train_index, test_index in folds.split(x,y):
    train_data = word_count_dframe.iloc[train_index]
    x_test_data = x.iloc[test_index]
    y_test_data = y.iloc[test_index]
    no_lap_res, lap_res = fit_and_predict(train_data, x_test_data, y_test_data)
    total_no_lap = [i+j for i, j in zip(no_lap_res, total_no_lap)]
    total_lap = [i+j for i, j in zip(lap_res, total_lap)]
    
print("----------------Total Results without Laplace-----------------")
print("True Positive:" + str(total_no_lap[0]) + " True Negative:" + str(total_no_lap[1]))
print("False Positive:" + str(total_no_lap[2]) + " False Negative:" + str(total_no_lap[3]))
print("Accuracy: " + str((total_no_lap[0] + total_no_lap[1]) / sum(total_no_lap)))
print("Precision: " + str(total_no_lap[0] / (total_no_lap[0] + total_no_lap[2])))
print("Specificity: " + str(total_no_lap[1] / (total_no_lap[1] + total_no_lap[2])))
print("Recall: " + str(total_no_lap[0] / (total_no_lap[0] + total_no_lap[3])))

print("----------------Total Results with Laplace-----------------")
print("True Positive:" + str(total_lap[0]) + " True Negative:" + str(total_lap[1]))
print("False Positive:" + str(total_lap[2]) + " False Negative:" + str(total_lap[3]))
print("Accuracy: " + str((total_lap[0] + total_lap[1]) / sum(total_lap)))
print("Precision: " + str(total_lap[0] / (total_lap[0] + total_lap[2])))
print("Specificity: " + str(total_lap[1] / (total_lap[1] + total_lap[2])))
print("Recall: " + str(total_lap[0] / (total_lap[0] + total_lap[3])))


________Without_laplace_______
Accuracy: 0.9497757847533632
True Positive: 104 True Negative:955
False Positive: 11 False Negative:45
Precision: 0.9043478260869565
Speficity: 0.9886128364389234
Recall: 0.697986577181208
__________laplace__________
Accuracy: 0.9623318385650225
True Positive: 107 True Negative:966
False Positive: 0 False Negative:42
Precision: 1.0
Speficity: 1.0
Recall: 0.7181208053691275



________Without_laplace_______
Accuracy: 0.9479820627802691
True Positive: 103 True Negative:954
False Positive: 12 False Negative:46
Precision: 0.8956521739130435
Speficity: 0.9875776397515528
Recall: 0.6912751677852349
__________laplace__________
Accuracy: 0.9542600896860987
True Positive: 98 True Negative:966
False Positive: 0 False Negative:51
Precision: 1.0
Speficity: 1.0
Recall: 0.6577181208053692



________Without_laplace_______
Accuracy: 0.9542600896860987
True Positive: 104 True Negative:960
False Positive: 6 False Negative:45
Precision: 0.9454545454545454
Speficity: 0.9937

We have generated 10 folds with 20% test data in each fold. Above cell contains the comparative stats for each fold, when using laplace smoothing and when not using laplace smoothing. In general we see that using laplace improves the perormance across all the metrices. The precision and speficity are almost 100% for all the folds.

We observe that by using laplace, following improvements in the overall metrics can be seen:

Accuracy: 0.9502242152466368 -> 0.9616143497757847

Precision: 0.9019776440240757 -> 0.99812382739212

Specificity: 0.9881987577639751 -> 0.9997929606625259

Recall: 0.7040268456375839 -> 0.7140939597315437

**Q4: Write a paragraph the summarizes the results and your thoughts about Naive Bayes classi-
cation for this problem**

We see that since there are around 86.6% of non-spam messages and remaining spam message. If we predict all the messages as non-spam, we will still achieve the accuracy of 86.6%. So the accuracy is not the right measure here. We want to find out the most of the spam messages.

We want all of the message which are classified as spam, to be spam. Because if an important message is classified as spam, in that case user will miss the email. User seeing a spam message, because our classifier could not classify as spam, does not impact as much as missing email does.

So our metric for classifier should be (correctly predicted spam) / (all predicted spam) = precision.

We see that bayesian classifier without laplace has very high accuracy, which is ~95% for all the folds. Even the precision is very high roughly ~90%. On examining the cases where the spam message classification wrongly predicted "Spam" or "Not spam", we see that in most cases it is because of presence of some specific words(gramatical errors) that are only present in spam or non-spam messages and not in both; examples include have -> hv, digits etc. In test data when we encounter such words, the probability immediately reduces to zero irrespective of what other set of words are pointing towards. With laplace smoothing this problems is resolved by not reducing the probability immediately to zero.

We see that laplace smoothing takes precision to even higher level(almost 100%) than traditional bayesian classifier.

Even though baysian assumption that words are not correlated is not true always. However making this general assumption, yields impressive results in our spam filter. 